In [ ]:
2
126, 101

8
140

6
86, 88, 112

5
31, 46

In [ ]:
import rasterio
from rasterio.windows import Window
import pandas as pd

# Function to check if a point is within the bounds of a rectangle
def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

# Read the CSV file with coordinates
coordinates_df = pd.read_csv('G:/Yehmh/DNDF/DNDF_PlantLine_twd97.csv')



# Define folder paths
tif_files = []
for i in range(2, 10):
    tif_files.append(f"G:/Yehmh/DNDF/202309_DNDF/pix4d file/00{i}/00{i}_transparent_mosaic_group1.tif")
for i in range(10, 16):
    tif_files.append(f"G:/Yehmh/DNDF/202309_DNDF/pix4d file/0{i}/0{i}_transparent_mosaic_group1.tif")

# Iterate through each TIF file
for tif_file in tif_files:
    with rasterio.open(tif_file) as src:
        transform = src.transform
        crs = src.crs
        res = src.res[0]  # assuming square pixels
        
        for index, row in coordinates_df.iterrows():
            x_coord = row['TWD97_X']
            y_coord = row['TWD97_Y']
            # Check if the coordinates fall within the bounds of the TIF file
            if point_inside_polygon(x_coord, y_coord, [(src.bounds.left, src.bounds.top), (src.bounds.right, src.bounds.top), (src.bounds.right, src.bounds.bottom), (src.bounds.left, src.bounds.bottom)]):
                # Convert the coordinates to pixel coordinates
                px, py = ~transform * (x_coord, y_coord)
                # Calculate the window size for 5x5 meters block
                window_size = 5 / res  # number of pixels for 5 meters

                if row['Plot'] in [26, 146]:
                    try:
                        for i in range(20):
                            for j in range(4):
                                cropx = px + j*window_size
                                cropy = py - window_size - i*window_size
                                # Define the window for cropping
                                window = Window(cropx, cropy, window_size, window_size)
                                # Read the crop
                                crop = src.read(window=window)
                                # Write the crop to a new TIF file
                                print(row['Plot'], row['Transect'], i, j)
                                with rasterio.open('G:/Yehmh/DNDF/202309_DNDF/belt_tiffs_2/{}_{}_{}_{}.tif'.format(row['Plot'], row['Transect'], i, j), 'w', driver='GTiff', width=crop.shape[2], height=crop.shape[1], count=src.count, dtype=crop.dtype, crs=crs, transform=src.window_transform(window)) as dst:
                                    dst.write(crop)
                    except rasterio.errors.RasterioIOError as e:
                        print(f"Error processing {tif_file}: {e}")
                        # Skip to the next iteration of the loop
                        continue
                else:
                    try:
                        for i in range(4):
                            for j in range(20):
                                try:
                                    cropx = px + j*window_size
                                    cropy = py - window_size - i*window_size
                                    # Define the window for cropping
                                    window = Window(cropx, cropy, window_size, window_size)
                                    # Read the crop
                                    crop = src.read(window=window)
                                    # Write the crop to a new TIF file
                                    print(row['Plot'], row['Transect'], i, j)
                                    with rasterio.open('G:/Yehmh/DNDF/202309_DNDF/belt_tiffs_2/{}_{}_{}_{}.tif'.format(row['Plot'], row['Transect'], i, j), 'w', driver='GTiff', width=crop.shape[2], height=crop.shape[1], count=src.count, dtype=crop.dtype, crs=crs, transform=src.window_transform(window)) as dst:
                                        dst.write(crop)
                                except rasterio.errors.RasterioIOError as e:
                                    print(f"Error processing {tif_file}: {e}")
                                    # Skip to the next iteration of the loop
                                    continue


In [ ]:
import rasterio
from rasterio.windows import Window
import pandas as pd
import os

# Function to check if a point is within the bounds of a rectangle
def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

# Read the CSV file with coordinates
coordinates_df = pd.read_csv('G:/Yehmh/DNDF/DNDF_PlantLine_twd97.csv')

# Define folder paths
tif_files = []
for i in range(2, 10):
    tif_files.append(f"G:/Yehmh/DNDF/202309_DNDF/pix4d file/00{i}/00{i}_transparent_mosaic_group1.tif")
for i in range(10, 16):
    tif_files.append(f"G:/Yehmh/DNDF/202309_DNDF/pix4d file/0{i}/0{i}_transparent_mosaic_group1.tif")

# Iterate through each TIF file
for tif_file in tif_files:
    with rasterio.open(tif_file) as src:
        transform = src.transform
        crs = src.crs
        res = src.res[0]  # assuming square pixels
        
        for index, row in coordinates_df.iterrows():
            x_coord = row['TWD97_X']
            y_coord = row['TWD97_Y']
            # Check if the coordinates fall within the bounds of the TIF file
            if point_inside_polygon(x_coord, y_coord, [(src.bounds.left, src.bounds.top), (src.bounds.right, src.bounds.top), (src.bounds.right, src.bounds.bottom), (src.bounds.left, src.bounds.bottom)]):
                # Convert the coordinates to pixel coordinates
                px, py = ~transform * (x_coord, y_coord)
                # Calculate the window size for 5x5 meters block
                window_size = 5 / res  # number of pixels for 5 meters

                if row['Plot'] in [26, 146]:
                    try:
                        for i in range(20):
                            for j in range(4):
                                cropx = px + j*window_size
                                cropy = py - window_size - i*window_size
                                # Define the window for cropping
                                window = Window(cropx, cropy, window_size, window_size)
                                # Read the crop
                                crop = src.read(window=window)
                                # Write the crop to a new TIF file
                                print(row['Plot'], row['Transect'], i, j)
                                
                                filename = 'G:/Yehmh/DNDF/202309_DNDF/belt_tiffs_2/{}_{}_{}_{}.tif'.format(row['Plot'], row['Transect'], i, j)

                                # Initialize the index
                                index = 0

                                # Check if the base filename already exists
                                while os.path.exists(filename):
                                    # If the base filename exists, modify the filename with an index appended to it
                                    index += 1
                                    filename = 'G:/Yehmh/DNDF/202309_DNDF/belt_tiffs_2/{}_{}_{}_{}({}).tif'.format(row['Plot'], row['Transect'], i, j, index)

                                # Open the file for writing using the found filename
                                with rasterio.open(filename, 'w', driver='GTiff', width=crop.shape[2], height=crop.shape[1], count=src.count, dtype=crop.dtype, crs=crs, transform=src.window_transform(window)) as dst:
                                    dst.write(crop)


                    except rasterio.errors.RasterioIOError as e:
                        print(f"Error processing {tif_file}: {e}")
                        # Skip to the next iteration of the loop
                        continue
                else:
                    try:
                        for i in range(4):
                            for j in range(20):                                
                                cropx = px + j*window_size
                                cropy = py - window_size - i*window_size
                                # Define the window for cropping
                                window = Window(cropx, cropy, window_size, window_size)
                                # Read the crop
                                crop = src.read(window=window)
                                # Write the crop to a new TIF file
                                print(row['Plot'], row['Transect'], i, j)
                                
                                filename = 'G:/Yehmh/DNDF/202309_DNDF/belt_tiffs_2/{}_{}_{}_{}.tif'.format(row['Plot'], row['Transect'], i, j)

                                # Initialize the index
                                index = 0

                                # Check if the base filename already exists
                                while os.path.exists(filename):
                                    # If the base filename exists, modify the filename with an index appended to it
                                    index += 1
                                    filename = 'G:/Yehmh/DNDF/202309_DNDF/belt_tiffs_2/{}_{}_{}_{}({}).tif'.format(row['Plot'], row['Transect'], i, j, index)

                                # Open the file for writing using the found filename
                                with rasterio.open(filename, 'w', driver='GTiff', width=crop.shape[2], height=crop.shape[1], count=src.count, dtype=crop.dtype, crs=crs, transform=src.window_transform(window)) as dst:
                                    dst.write(crop)

                    except rasterio.errors.RasterioIOError as e:
                        print(f"Error processing {tif_file}: {e}")
                        # Skip to the next iteration of the loop
                        continue